## Basic atomese examples

In [1]:
from opencog.utilities import initialize_opencog
from opencog.atomspace import AtomSpace, types
from opencog.type_constructors import *
from opencog.bindlink import bindlink
from opencog.bindlink import execute_atom


/home/noskill/miniconda3/envs/pmvqa3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'opencog.utilities' does not match runtime version 3.5
  return f(*args, **kwds)
/home/noskill/miniconda3/envs/pmvqa3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'opencog.atomspace' does not match runtime version 3.5
  return f(*args, **kwds)
/home/noskill/miniconda3/envs/pmvqa3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'opencog.type_constructors' does not match runtime version 3.5
  return f(*args, **kwds)
/home/noskill/miniconda3/envs/pmvqa3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'opencog.bindlink' does not match runtime version 3.5
  return f(*args, **kwds)


Make a new atomspace  
Atomspace is a hypergraph database  
which can store nodes, links and associated values

In [2]:
atomspace = AtomSpace()

add concept node  
Nodes destingueshed by type and name and are immutable

In [3]:
atomspace.add_node(types.ConceptNode, "small")

(ConceptNode "small") ; [8595782527577120975][1]

set default atomspace to add atoms to

In [4]:
initialize_opencog(atomspace)


adding node one more time doesn't modify anything

In [5]:
ConceptNode("small")

(ConceptNode "small") ; [8595782527577120975][1]

every Atom(nodes or links) has incoming and outgoing sets

In [6]:
print('ConceptNode("size").incoming:')
print(ConceptNode("size").incoming)

ConceptNode("size").incoming:
[]


every Atom has default tv:

In [7]:
link = InheritanceLink(ConceptNode("small"), ConceptNode("size"))
print("create link {0}\n".format(link))

print(link.tv)

create link (InheritanceLink
  (ConceptNode "small")
  (ConceptNode "size")
)


(stv 1.000000 0.000000)


setting new TruthValue

In [8]:
link.tv = TruthValue(0.9, 0.9)

Outgoing set of a link is set of Atoms it connects

In [9]:
print(link)
print('outgoing set of the link:')
print(link.out)

(InheritanceLink (stv 0.900000 0.900000)
  (ConceptNode "small")
  (ConceptNode "size")
)

outgoing set of the link:
[(ConceptNode "small") ; [8595782527577120975][1]
, (ConceptNode "size") ; [7908756497414962438][1]
]


Let's put the InheritanceLink into a ListLink  
ListLink is just ordered List of atoms

In [10]:
ListLink(link)

(ListLink
  (InheritanceLink (stv 0.900000 0.900000)
    (ConceptNode "small") ; [8595782527577120975][1]
    (ConceptNode "size") ; [7908756497414962438][1]
  ) ; [16030665450786561781][1]
) ; [14106166215360076800][1]

In [18]:
print('incoming set of the link:')
print(link.incoming)

incoming set of the link:
[(SetLink
  (InheritanceLink (stv 0.900000 0.900000)
    (ConceptNode "small") ; [8595782527577120975][1]
    (ConceptNode "size") ; [7908756497414962438][1]
  ) ; [16030665450786561781][1]
) ; [14110089272848010777][1]
, (ListLink
  (InheritanceLink (stv 0.900000 0.900000)
    (ConceptNode "small") ; [8595782527577120975][1]
    (ConceptNode "size") ; [7908756497414962438][1]
  ) ; [16030665450786561781][1]
) ; [14106166215360076800][1]
]


While incoming set of Atom is set of links pointing to it

In [12]:
print('\n incoming set of ConceptNode("size")')
print(ConceptNode("size").incoming)


 incoming set of ConceptNode("size")
[(InheritanceLink (stv 0.900000 0.900000)
  (ConceptNode "small") ; [8595782527577120975][1]
  (ConceptNode "size") ; [7908756497414962438][1]
) ; [16030665450786561781][1]
]


Thus outgoing set of Nodes are empty

In [13]:
ConceptNode("size").out

[]

Let's take a look at BindLink  
Semantics of BindLink is  
"do something if pattern matches and return the result"

**Bindlink structure**
1. VariableList - optional  
2. Conditional part - pattern to match
3. Rewrite term  

Our condition will be InheritanceLink between  two variables  
So pattern matcher will try to replace VariableNodes with other Atoms

In [14]:
condition = InheritanceLink(VariableNode("X"), VariableNode("Y"))

Let's return the same link which is matched in condition

In [15]:
bindlink1 = BindLink(condition, condition)
print(str(bindlink1))

(BindLink
  (InheritanceLink
    (VariableNode "X")
    (VariableNode "Y")
  )
  (InheritanceLink
    (VariableNode "X")
    (VariableNode "Y")
  )
)



BindLink constructor doesn't execute anything  
There is interface function for pattern matcher  
"bindlink" which accepts atomspace to apply bindlink to  
and bindlink itself

In [16]:
print("running bindlink")
print("result: \n{0}".format(bindlink(atomspace, bindlink1)))

running bindlink
result: 
(SetLink
  (InheritanceLink (stv 0.900000 0.900000)
    (ConceptNode "small")
    (ConceptNode "size")
  )
)



______________________________________________________
Another usefull type of link is ExecutionOutputLink  
which used to run callback function on matched pattern  
ExecutionOutputLink should return Atom


**ExecutionOutputLink structure**
1. GroundedSchemaNode
2. ListLink of arguments

let's define a callback function which returns a ConceptNode

In [20]:
def add_node():
    return ConceptNode("Test")

Now GroundedSchemaNode which accepts string of following format  
"&lt;language&gt;:&lt;function name&gt;"


where language can be any of "lib", "py", "scm"

In case of lib atomspace will try to open library with dlopen  
so any shared library may be used

In [21]:
add_node_schema = GroundedSchemaNode("py: add_node")

Now make Execution output link

In [23]:
exec1 = ExecutionOutputLink(
         add_node_schema,
         ListLink())


ExecutionOutputLink may be used inside of bindlink or  
it may be executed by execute_atom function

In [24]:
print("running ExecutionOutputLink")
print(execute_atom(atomspace, exec1))

running ExecutionOutputLink
(ConceptNode "Test")



ExecutionOutputLinks may be nested(actually any links may)

In [26]:
print("nested ExectutionOutputLinks")

def add_link(node):
    ev = EvaluationLink(
           PredicateNode("ok"),
           node)
    ev.tv = TruthValue(0.1, 0.8)
    return ev

exec2 = ExecutionOutputLink(
           GroundedSchemaNode("py: add_link"),
           ListLink(
             ExecutionOutputLink(
                 GroundedSchemaNode("py: add_node"),
                 ListLink()))
        )

print(execute_atom(atomspace, exec2))



nested ExectutionOutputLinks
(EvaluationLink (stv 0.100000 0.800000)
  (PredicateNode "ok")
  (ConceptNode "Test")
)



Exercise: print incoming set of (InheritanceLink (ConceptNode "small") (ConceptNode "size") )  
once more time